In [2]:
from typing import Dict , TypedDict , Optional
from langgraph.graph import StateGraph , END
import random
import time
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import CommaSeparatedListOutputParser 
import os
from langchain.output_parsers import OutputFixingParser

llm = ChatOpenAI()

In [12]:
class GraphState(TypedDict):
    history: Optional[str]= None
    result:Optional[str]= None
    total_question:Optional[str]= None
    interviewer:Optional[str]= None
    candidate:Optional[str]= None
    current_question:Optional[str]= None
    current_answer:Optional[str]= None
        
workflow = StateGraph(GraphState)

In [4]:
prompt_interviewer = "You are a {}. You need to interview a {}. This is the interview so far:\n{}\n\
Ask your next question and donot repeat Question.\
Keep it less than 10 words and output just the question and no extra text"

prompt_interviewee = "You are a {}. You have appeared for a job interview.\
Answer the question asked in very short in less than 10 words. Outputs Just the answer with no extra text.\
Question:{}"

prompt_result = """
Check weather the answer given for asked question is correct or not?
Evaluate on a Scale of 10 and give a very sort (maximum 10 words) reason as well
question:{}\nanswer:{}"""

prompt_verdict = """Given the interview, should we select the candidate?
Give output Yes or NO with a reason in less than 10 words
the interview:{}"""

prompt_cleanup = "Remove empty dialogues , repeated sentences and repeted names to convert this input as conversation:\n{}"

In [5]:
def handle_question(state):
    history = state.get("history","").strip()
    role = state.get("interviewer","").strip()
    candidate = state.get("candidate","").strip()
    
    
    prompt = prompt_interviewer.format(role , candidate, history)
    
    print(prompt)
    
    question = role +":"+ llm(prompt)
    
    print("Question:",question)
    
    if history =="Nothing":
        history= ""
        
    return {"history":history +"\n" + question,
           "current_question":question,
           "total_question":state.get("total_questions")+1}

In [16]:
def handle_response(state, user_input):
    history = state.get("history","").strip()
    role = state.get("interviewer","").strip()
    candidate = state.get("candidate","").strip()
    
    prompt = prompt_interviewee.format(candidate,question)
    print(prompt)
    
    answer = candidate + ":" + user_input
    
    print("Response:",answer)
    
    return {"history":history +"\n" + answer,
           "current_question":answer}
    

In [8]:
def handle_evaluate(state):
    question = state.get("current_question","").strip()
    answer = state.get("current_answer","").strip()
    history = state.get("history","").strip()
    
    
    prompt = prompt_result.format(question , answer)
    evaluation = llm(prompt)
    print(prompt)
    
    
    print("Evaluation:",evaluation)
    
    print("--------------DONE--------------")
    
    return {"history":histroy}+"\n"+evaluation


In [10]:
def handle_result(state):
    history = state.get("history","").strip()
    cleaned_up = llm(prompt_cleanup.format(history))
    prompt = prompt_verdict.format(cleaned_up)
    
    result - llm(prompt)
    
    print(prompt)
    
    print("result:",result)
    
    return{"result":result,"histroy":cleaned_up}

In [13]:
workflow.add_node("handle_question",handle_question)
workflow.add_node("handle_evaluate",handle_evaluate)
workflow.add_node("handle_response",handle_response)
workflow.add_node("handle_result",handle_result)

In [14]:
def check_conv_length(state):
    return "handle_question" if state.get("total_question")<3 else "handle_result"

In [15]:
workflow.add_conditional_edges(
"handle_evaluate",
check_conv_length,
{
    "handle_question","handle_question",
    "handle_evaluate","handle_evaluate",
    "handle_response","handle_response",
    "handle_result",'handle_result',
    
})

In [ ]:
workflow.set_entry_point("handle_question")